In [ ]:
using CSV
using DataFrames
using CairoMakie
using ColorSchemes

const global_scheme = ColorSchemes.seaborn_pastel

In [ ]:
function plot_latencies()
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    ax_udp = Axis(fig[1, 1], xscale=log2, yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Latency [μs]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    udp_data = []
    qsfp_data = []
    base_data = []
    for dft in groupby(df[df.protocol .== "UDP",:],"type")
        if occursin("MPI+PCIe", first(dft[!,"type"]))
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "latency"] * 1.0e6, label=first(dft[!,"type"]), linestyle=:dot)
            push!(base_data, d)
        else
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "latency"] * 1.0e6, label="$(first(dft[!,"type"])) Byte", linestyle=:dash)
            push!(udp_data, d)
        end
    end
    df_cpu = sort(df[(df.protocol .== "base") .&& (df.type .== "CPU"),:], "MSize")
    push!(base_data, lines!(ax_udp, df_cpu[!, "MSize"], df_cpu[!, "latency"] * 1.0e6, label="MPI", linestyle=:dot))
    
    df_aurora_streaming = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Streaming"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_streaming[!, "MSize"], df_aurora_streaming[!, "latency"] * 1.0e6, label="Aurora/Streaming", linestyle=:solid))

    df_aurora_framing = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Framing"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_framing[!, "MSize"], df_aurora_framing[!, "latency"] * 1.0e6, label="Aurora/Framing", linestyle=:solid))

    df_seriallite = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "SerialLite"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_seriallite[!, "MSize"], df_seriallite[!, "latency"] * 1.0e6, label="SerialLite (2x 40G)", linestyle=:solid))
    
    xlims!(ax_udp, (2^6, 2^20))
    ylims!(ax_udp, (10^-1, 10^4))
    Legend(fig[1, 2], [udp_data, base_data, qsfp_data], [[u.label for u in d] for d in [udp_data, base_data, qsfp_data]], ["UDP Max. Payload", "Baseline", "Circuit-Switched"], nbanks=1, orientation=:vertical)
    fig
end

plot_latencies()

In [ ]:
function plot_throughput()
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    ax_udp = Axis(fig[1, 1], xscale=log2, yscale=identity,
        xlabel="Message Size [Byte]", ylabel="Throughput [Gbit/s]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    udp_data = []
    qsfp_data = []
    base_data = []

    df[!, "throughput"] = df[!, "MSize"] ./ df[!, "latency"] * 8 / 1000000000
    
    for dft in groupby(df[df.protocol .== "UDP",:],"type")
        if occursin("MPI+PCIe", first(dft[!,"type"]))
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "throughput"], label=first(dft[!,"type"]), linestyle=:dot)
            push!(base_data, d)
        else
            d = lines!(ax_udp, dft[!, "MSize"], dft[!, "throughput"], label=first(dft[!,"type"]), linestyle=:dash)
            push!(udp_data, d)
        end
    end
    df_cpu = sort(df[(df.protocol .== "base") .&& (df.type .== "CPU"),:], "MSize")
    push!(base_data, lines!(ax_udp, df_cpu[!, "MSize"], df_cpu[!, "throughput"], label="MPI", linestyle=:dot))
    
    df_aurora_streaming = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Streaming"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_streaming[!, "MSize"], df_aurora_streaming[!, "throughput"], label="Aurora/Streaming", linestyle=:solid))

    df_aurora_framing = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "Aurora/Framing"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_aurora_framing[!, "MSize"], df_aurora_framing[!, "throughput"], label="Aurora/Framing", linestyle=:solid))

    df_seriallite = sort(df[(df.protocol .== "QSFP") .&& (df.type .== "SerialLite"),:], "MSize")
    push!(qsfp_data, lines!(ax_udp, df_seriallite[!, "MSize"], df_seriallite[!, "throughput"], label="SerialLite (2 x 40G)", linestyle=:solid))
    
    xlims!(ax_udp, (2^6, 2^20))
    ylims!(ax_udp, (10^-1, 10^2))
    Legend(fig[1, 2], [udp_data, base_data, qsfp_data], [[u.label for u in d] for d in [udp_data, base_data, qsfp_data]], ["ACCL", "Baseline", "Circuit-Switched"], nbanks=1, orientation=:vertical)
    fig
end

plot_throughput()

In [ ]:
function plot_over_message_size(message_size)
    df = CSV.read("data.csv", DataFrame)
    fig = Figure()
    df = sort(df[(df.MSize .== message_size),:], rev=true)
    delete!(df, 1)
    barplot(
        fig[1,1],
        df[!, "latency"] * 1.0e6,
        bar_labels = :y,
        axis=(
            xlabel="Communication Type", ylabel="Latency [μs]",
            yminorticksvisible=true, yminorgridvisible=true,
            yminorticks=IntervalsBetween(5),
            xticks = (1:5, ["ACCL PL Stream", "MPI", "Aurora\nFraming", "Aurora\nStreaming", "SerialLite"]),
        ),
    )
    fig
end

plot_over_message_size(64)

In [ ]:
function plot_throughput_over_frame_sizes()
    df = CSV.read("latency_over_frame_sizes.csv", DataFrame)
    df_udp = CSV.read("data.csv", DataFrame)
    filter!(x -> x.protocol == "UDP" && x.type != "MPI+PCIe", df_udp)
    df_udp[!, "throughput"] = df_udp[!, "MSize"] ./ df_udp[!, "latency"] * 8 / 1000000000
    sort!(df_udp, :type, by=x->parse(Int,x))
    fig = Figure()
    ax = Axis(fig[1, 1], xscale=log2, #yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Throughput [Gbit/s]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    data = []
    frame_sizes = [2,4,7,10]
    message_sizes = df[:, 1]

    udp_data = [] 
    types = sort(unique(df_udp[!,"type"]), by=x->parse(Int,x))
    typecolor = Dict(t => global_scheme[i] for (i,t) in enumerate(types))
    for (i, dft) in enumerate(groupby(df_udp,"type"))
        d = lines!(ax, dft[!, "MSize"], dft[!, "throughput"], label="$(first(dft[!,"type"])) Byte", linestyle=:dash, color=typecolor[first(dft[!,"type"])])
        push!(udp_data, d)
    end

    for (c,i) in enumerate(frame_sizes)
        frame_size = 2^i
        df_frame = dropmissing(df[!, ["message_size", string(frame_size)]])
        valid_message_sizes = 2.0 .^(i:i+nrow(df_frame)-1) .* 64
        valid_df_frame = valid_message_sizes ./ df_frame[!,string(frame_size)] .* 8 ./ 1000000000
        #display(valid_df_frame)
        push!(data, lines!(ax, valid_message_sizes, valid_df_frame, label="$(frame_size * 64) Byte", color=global_scheme[c]))
    end
    
    xlims!(ax, (2^6, 2^25))
    ylims!(ax, (10^0, 10^2))
    Legend(fig[1, 2], [data, reverse(udp_data)], [[l.label for l in data], reverse([l.label for l in udp_data])], ["Frame Size", "UDP Payload"], nbanks=1, orientation=:vertical)
    fig
end

plot_throughput_over_frame_sizes()

In [ ]:
function plot_latencies_over_frame_sizes()
    df = CSV.read("latency_over_frame_sizes.csv", DataFrame)
    fig = Figure()
    ax = Axis(fig[1, 1], xscale=log2, yscale=log10,
        xlabel="Message Size [Byte]", ylabel="Latency [μs]",
        yminorticksvisible=true, yminorgridvisible=true,
        yminorticks=IntervalsBetween(5),
        xminorticksvisible=true, xminorgridvisible=true,
        xminorticks=IntervalsBetween(5),
        palette=(color=global_scheme,))
    data = []
    frame_sizes = [2^n for n in 0:22]
    message_sizes = df[:, 1]

    for (i, frame_size) in enumerate(frame_sizes)
        df_frame = df[:, i+1]
        valid_indices = findall(!ismissing, df_frame)
        valid_message_sizes = message_sizes[valid_indices]
        valid_df_frame = Float64.(df_frame[valid_indices]) .* 1e+6
        #display(valid_df_frame)
        push!(data, lines!(ax, valid_message_sizes, valid_df_frame, label=string(frame_size)))
    end
    
    xlims!(ax, (2^6, 2^20))
    ylims!(ax, (10^0, 10^2))
    Legend(fig[1, 2], data, [l.label for l in data], ["Frame Size"], nbanks=1, orientation=:vertical)
    fig
end

plot_latencies_over_frame_sizes()